In [1]:
dbutils.widgets.text("movie_data", defaultValue="dbfs:/FileStore/movielens/test-data/movies.json", label="Movie source path")
dbutils.widgets.text("ratings_data", defaultValue="dbfs:/FileStore/movielens/test-data/ratings.json", label="Ratings source path")
dbutils.widgets.text("target_table", defaultValue="test_load_movies", label="Output movie table name")

In [2]:
movie_data = dbutils.widgets.get("movie_data")
ratings_data = dbutils.widgets.get("ratings_data")
target_table = dbutils.widgets.get("target_table")

In [3]:
movies_df = (spark
             .read
             .json(movie_data)
             .select('movieId', 'title')
            )
display(movies_df)

movieId,title
9999,Unknow within
6548,Bad Boys II (2003)
145,Bad Boys (1995)


In [4]:
ratings_df = (spark
             .read
             .json(ratings_data)
             .select('movieId', 'rating')
            )
display(ratings_df)

movieId,rating
145,4.0
145,3.0
6548,2.0
999999,5.0


In [5]:
movies_with_rating_df = (movies_df
             .join(ratings_df, ['movieId'], how='full'))
display(movies_with_rating_df)

movieId,title,rating
999999,null,5.0
145,Bad Boys (1995),4.0
145,Bad Boys (1995),3.0
9999,Unknow within,null
6548,Bad Boys II (2003),2.0


In [6]:
import pyspark.sql.functions as F

target_df = (movies_with_rating_df
                         .groupBy('movieId', 'title')
                         .agg(
                           F.min('rating').alias('min_rating'),
                           F.max('rating').alias('max_rating'),
                           F.count('rating').alias('num_ratings'),
                           F.round(F.sum('rating')/F.count('rating'), 1).alias('average_rating')                           
                         )
                        )
display(target_df)

movieId,title,min_rating,max_rating,num_ratings,average_rating
999999,null,5.0,5.0,1,5.0
9999,Unknow within,null,null,0,null
145,Bad Boys (1995),3.0,4.0,2,3.5
6548,Bad Boys II (2003),2.0,2.0,1,2.0


In [7]:
(target_df
  .write
  .mode("overwrite")
  .saveAsTable(target_table))